##### 版權 2024 Google LLC.


In [1]:
#@title Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
#
# https://www.apache.org/licenses/LICENSE-2.0
#
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.

In [2]:
# The non-source code materials on this page are licensed under Creative Commons - Attribution-ShareAlike CC-BY-SA 4.0,
# https://creativecommons.org/licenses/by-sa/4.0/legalcode.

# ReAct + Gemini：一種提示方法用於示範 LLM 中的推理和行動


<table class="tfo-notebook-buttons" align="left">
  <td>
    <a target="_blank" href="https://ai.google.dev/docs/react_gemini_prompting"><img src="https://ai.google.dev/static/site-assets/images/docs/notebook-site-button.png" height="32" width="32" />在 ai.google.dev 上檢視</a>
  </td>
  <td>
    <a target="_blank" href="https://colab.research.google.com/github/doggy8088/generative-ai-docs/blob/main/site/zh/docs/react_gemini_prompting.ipynb"><img src="https://www.tensorflow.org/images/colab_logo_32px.png" />在 Google Colab 中執行</a>
  </td>
  <td>
    <a target="_blank" href="https://github.com/doggy8088/generative-ai-docs/blob/main/site/zh/docs/react_gemini_prompting.ipynb"><img src="https://www.tensorflow.org/images/GitHub-Mark-32px.png" />在 GitHub 上檢視原始程式碼</a>
  </td>
</table>


這個筆記本是 [ReAct: Synergizing Reasoning and Acting in Language Models](https://arxiv.org/abs/2210.03629) 的最小實作，使用了一款來自 Google 的 `gemini-pro` 模型。


這個筆記本示範了如何使用 `gemini-pro` 透過利用 **少樣本 ReAct 提示** 來產生推理追蹤和針對特定任務的動作。  在這個逐步說明中，你會學習執行下列事項：


1. 設定開發環境和 API 存取權以使用 Gemini。
2. 以 ReAct 提示 Gemini。
3. 將新提示的模型用於多回合對話 (聊天)。
4. ReAct 如何藉由透過 **維基百科 API** 尋求外部地面真實，來克服幻覺和錯誤傳播的問題。
5. 與已部署的 **ReAct 提示的 Gemini 機器人 🤖** 進行對話


### 背景


ReAct ((姚等人，2022 年)[https://arxiv.org/abs/2210.03629]) 是一種提示方式，這允許多語言模型追蹤推理步驟，涉及回答使用者的查詢。這提升了人類可解釋性和信賴度。對 ReAct 進行提示的模型為每次疊代產生 **想法-動作-觀察** 三重項。


與其指示模型「逐步驟說明」，使用 ReAct 鼓勵模型透過誘發想法和動作步驟來尋找事實資訊，並使用外部工具提供觀察步驟。


這會透過增加對應特定動作的提示結構來運作。

      - 搜尋[實體]：搜尋特定 `實體` (在此指南中，它將查詢 Wikipedia API)。
      - 檢索[片語]：掃描 `搜尋` 結果，找尋特定、完全相符的片語 (在 Wikipedia 的頁面上)。
      - 完成[答案]：將 `答案` 回傳給使用者。


## 設定


### 安裝 Python SDK

Gemini API 的 Python SDK 包含於 [`google-generativeai`](https://pypi.org/project/google-generativeai/) 套件中。使用 pip 安裝相依性套件：

你還需要安裝 **Wikipedia** API。


In [1]:
!pip install -q google.generativeai

In [2]:
!pip install -q wikipedia

備註：[ `wikipedia` 套件](https://pypi.org/project/wikipedia/) 指出它「以簡單易用為設計，而非進階用途」，而生產或大量使用應「使用 [Pywikipediabot](http://www.mediawiki.org/wiki/Manual:Pywikipediabot) 或其他進階 [Python MediaWiki API 封裝](http://en.wikipedia.org/wiki/Wikipedia:Creating_a_bot#Python)」。


### 匯入套件


匯入必要的套件。


In [ ]:
import re
import os

import wikipedia
from wikipedia.exceptions import DisambiguationError, PageError

import google.generativeai as genai

### 設定你的 API 金鑰

在開始使用 Gemini API 前，你必須先取得 API 金鑰。如果你還沒有，請在 Google AI Studio 點一下建立金鑰。

<a class="button button-primary" href="https://makersuite.google.com/app/apikey" target="_blank" rel="noopener noreferrer">取得 API 金鑰</a>


在 Colab 中，在左側面板的「🔑」下新增秘鑰至秘鑰管理服務。將其命名為 `GOOGLE_API_KEY`。


一旦你取得了 API 金鑰，請將其傳遞到 SDK。你可以用兩種方式進行此事：

* 將金鑰放入 `GOOGLE_API_KEY` 環境變數中 (SDK 會自動從中撷取)。
* 將金鑰傳遞到 `genai.configure(api_key=...)`


In [7]:
try:
    from google.colab import userdata
    GOOGLE_API_KEY=userdata.get('GOOGLE_API_KEY')
except ImportError as e:
    GOOGLE_API_KEY = os.environ['GOOGLE_API_KEY']
genai.configure(api_key=GOOGLE_API_KEY)

## ReAct 輸入提示


這裡，你會與 [原始 ReAct 提示](https://github.com/ysymyth/ReAct/tree/master/prompts) 合作，做一些微小的調整。


> 備註：此處使用提示和情境範例均取自 [https://github.com/ysymyth/ReAct](https://github.com/ysymyth/ReAct)，其已在 [麻省理工學院許可證](https://opensource.org/licenses/MIT) 下發布。


In [8]:
model_instructions = """Solve a question answering task with interleaving Thought, Action, Observation steps. Thought can reason about the current situation, Observation is understanding relevant information from an Action's output and Action can be of three types:
(1) <search>entity</search>, which searches the exact entity on Wikipedia and returns the first paragraph if it exists. If not, it will return some similar entities to search and you can try to search the information from those topics.
(2) <lookup>keyword</lookup>, which returns the next sentence containing keyword in the current context. This only does exact matches, so keep your searches short.
(3) <finish>answer</finish>, which returns the answer and finishes the task.
"""

### 少樣本提示，以 Gemini 啟用情境內學習


雖然大型語言模型表現出對指令有良好的理解，但它們在不經過調整的情況下，仍可能在複雜任務上表現不佳。因此，你現在會提供一些範例，連同你的提示，以根據你的需要來引導模型輸出。這種**情境學習** 大幅提升了模型的性能。


In [9]:
examples = """
Here are some examples.

Question
What is the elevation range for the area that the eastern sector of the Colorado orogeny extends into?

Thought 1
I need to search Colorado orogeny, find the area that the eastern sector of the Colorado orogeny extends into, then find the elevation range of the area.

Action 1
<search>Colorado orogeny</search>

Observation 1
The Colorado orogeny was an episode of mountain building (an orogeny) in Colorado and surrounding areas.

Thought 2
It does not mention the eastern sector. So I need to look up eastern sector.

Action 2
<lookup>eastern sector</lookup>

Observation 2
The eastern sector extends into the High Plains and is called the Central Plains orogeny.

Thought 3
The eastern sector of Colorado orogeny extends into the High Plains. So I need to search High Plains and find its elevation range.

Action 3
<search>High Plains</search>

Observation 3
High Plains refers to one of two distinct land regions

Thought 4
I need to instead search High Plains (United States).

Action 4
<search>High Plains (United States)</search>

Observation 4
The High Plains are a subregion of the Great Plains. From east to west, the High Plains rise in elevation from around 1,800 to 7,000 ft (550 to 2,130m).

Thought 5
High Plains rise in elevation from around 1,800 to 7,000 ft, so the answer is 1,800 to 7,000 ft.

Action 5
<finish>1,800 to 7,000 ft</finish>

Question
Musician and satirist Allie Goertz wrote a song about the "The Simpsons" character Milhouse, who Matt Groening named after who?

Thought 1
The question simplifies to "The Simpsons" character Milhouse is named after who. I only need to search Milhouse and find who it is named after.

Action 1
<search>Milhouse</search>

Observation 1
Milhouse Mussolini Van Houten is a recurring character in the Fox animated television series The Simpsons voiced by Pamela Hayden and created by Matt Groening.

Thought 2
The paragraph does not tell who Milhouse is named after, maybe I can look up "named after".

Action 2
<lookup>named after</lookup>

Observation 2
Milhouse was named after U.S. president Richard Nixon, whose middle name was Milhous.

Thought 3
Milhouse was named after U.S. president Richard Nixon, so the answer is Richard Nixon.

Action 3
<finish>Richard Nixon</finish>

Question
Which documentary is about Finnish rock groups, Adam Clayton Powell or The Saimaa Gesture?

Thought 1
I need to search Adam Clayton Powell and The Saimaa Gesture, and find which documentary is about Finnish rock groups.

Action 1
<search>Adam Clayton Powell</search>

Observation 1
Could not find [Adam Clayton Powell]. Similar: [’Adam Clayton Powell III’, ’Seventh Avenue (Manhattan)’, ’Adam Clayton Powell Jr. State Office Building’, ’Isabel Washington Powell’, ’Adam Powell’, ’Adam Clayton Powell (film)’, ’Giancarlo Esposito’].

Thought 2
To find the documentary, I can search Adam Clayton Powell (film).

Action 2
<search>Adam Clayton Powell (film)</search>

Observation 2
Adam Clayton Powell is a 1989 American documentary film directed by Richard Kilberg. The film is about the rise and fall of influential African-American politician Adam Clayton Powell Jr.[3][4] It was later aired as part of the PBS series The American Experience.

Thought 3
Adam Clayton Powell (film) is a documentary about an African-American politician, not Finnish rock groups. So the documentary about Finnish rock groups must instead be The Saimaa Gesture.

Action 3
<finish>The Saimaa Gesture</finish>

Question
What profession does Nicholas Ray and Elia Kazan have in common?

Thought 1
I need to search Nicholas Ray and Elia Kazan, find their professions, then find the profession they have in common.

Action 1
<search>Nicholas Ray</search>

Observation 1
Nicholas Ray (born Raymond Nicholas Kienzle Jr., August 7, 1911 - June 16, 1979) was an American film director, screenwriter, and actor best known for the 1955 film Rebel Without a Cause.

Thought 2
Professions of Nicholas Ray are director, screenwriter, and actor. I need to search Elia Kazan next and find his professions.

Action 2
<search>Elia Kazan</search>

Observation 2
Elia Kazan was an American film and theatre director, producer, screenwriter and actor.

Thought 3
Professions of Elia Kazan are director, producer, screenwriter, and actor. So profession Nicholas Ray and Elia Kazan have in common is director, screenwriter, and actor.

Action 3
<finish>director, screenwriter, actor</finish>

Question
Which magazine was started first Arthur’s Magazine or First for Women?

Thought 1
I need to search Arthur’s Magazine and First for Women, and find which was started first.

Action 1
<search>Arthur’s Magazine</search>

Observation 1
Arthur’s Magazine (1844-1846) was an American literary periodical published in Philadelphia in the 19th century.

Thought 2
Arthur’s Magazine was started in 1844. I need to search First for Women next.

Action 2
<search>First for Women</search>

Observation 2
First for Women is a woman’s magazine published by Bauer Media Group in the USA.[1] The magazine was started in 1989.

Thought 3
First for Women was started in 1989. 1844 (Arthur’s Magazine) < 1989 (First for Women), so Arthur’s Magazine was started first.

Action 3
<finish>Arthur’s Magazine</finish>

Question
Were Pavel Urysohn and Leonid Levin known for the same type of work?

Thought 1
I need to search Pavel Urysohn and Leonid Levin, find their types of work, then find if they are the same.

Action 1
<search>Pavel Urysohn</search>

Observation 1
Pavel Samuilovich Urysohn (February 3, 1898 - August 17, 1924) was a Soviet mathematician who is best known for his contributions in dimension theory.

Thought 2
Pavel Urysohn is a mathematician. I need to search Leonid Levin next and find its type of work.

Action 2
<search>Leonid Levin</search>

Observation 2
Leonid Anatolievich Levin is a Soviet-American mathematician and computer scientist.

Thought 3
Leonid Levin is a mathematician and computer scientist. So Pavel Urysohn and Leonid Levin have the same type of work.

Action 3
<finish>yes</finish>

Question
{question}"""

將附有範例的說明複製到稱為 `model_instructions.txt` 的檔案中


In [10]:
ReAct_prompt = model_instructions + examples
with open('model_instructions.txt', 'w') as f:
  f.write(ReAct_prompt)

## 將 ReAct 與 Gemini 結合使用


### 設定


現在，你將建立一個類別，以利使用 ReAct 提示的 Gemini 模型進行多輪聊天。


In [11]:
class ReAct:
  def __init__(self, model: str, ReAct_prompt: str | os.PathLike):
    """Prepares Gemini to follow a `Few-shot ReAct prompt` by imitating
    `function calling` technique to generate both reasoning traces and
    task-specific actions in an interleaved manner.

    Args:
        model: name to the model.
        ReAct_prompt: ReAct prompt OR path to the ReAct prompt.
    """
    self.model = genai.GenerativeModel(model)
    self.chat = self.model.start_chat(history=[])
    self.should_continue_prompting = True
    self._search_history: list[str] = []
    self._search_urls: list[str] = []

    try:
      # try to read the file
      with open(ReAct_prompt, 'r') as f:
        self._prompt = f.read()
    except FileNotFoundError:
      # assume that the parameter represents prompt itself rather than path to the prompt file.
      self._prompt = ReAct_prompt

  @property
  def prompt(self):
    return self._prompt

  @classmethod
  def add_method(cls, func):
    setattr(cls, func.__name__, func)

  @staticmethod
  def clean(text: str):
    """Helper function for responses."""
    text = text.replace("\n", " ")
    return text

### 定義工具


按照提示，模型將生成 **思考-行動-觀察** 追蹤，其中每個 **行動** 追蹤皆可能是下列 Token 之一：


1.   <搜尋/>：透過外部 API 執行 Wikipedia 搜尋。
2.   <查找/>：使用 Wikipedia API 查詢頁面上的特定資訊。
3.   <完成/>：停止模型執行並傳回答案。

如果傳回其中任何一個 `行動` Token，你都必須呼叫相關工具，並使用 `觀察` 更新提示。

注意：Gemini API 支援 [函式呼叫](https://ai.google.dev/docs/function_calling)，你可以使用此功能設定工具。然而，針對本教學課程，你將以類似方式使用 `stop_sequences` 參數。


#### 搜尋
定義一種執行維基百科搜尋的方法


In [12]:
@ReAct.add_method
def search(self, query: str):
    """Perfoms search on `query` via Wikipedia api and returns its summary.

    Args:
        query: Search parameter to query the Wikipedia API with.

    Returns:
        observation: Summary of Wikipedia search for `query` if found else
        similar search results.
    """
    observation = None
    query = query.strip()
    try:
      # try to get the summary for requested `query` from the Wikipedia
      observation = wikipedia.summary(query, sentences=4, auto_suggest=False)
      wiki_url = wikipedia.page(query, auto_suggest=False).url
      observation = self.clean(observation)

      # if successful, return the first 2-3 sentences from the summary as model's context
      observation = self.model.generate_content(f'Retun the first 2 or 3 \
          sentences from the following text: {observation}')
      observation = observation.text

      # keep track of the model's search history
      self._search_history.append(query)
      self._search_urls.append(wiki_url)
      print(f"Information Source: {wiki_url}")

    # if the page is ambiguous/does not exist, return similar search phrases for model's context
    except (DisambiguationError, PageError) as e:
      observation = f'Could not find ["{query}"].'
      # get a list of similar search topics
      search_results = wikipedia.search(query)
      observation += f' Similar: {search_results}. You should search for one of those instead.'

    return observation

#### 查字
在維基百科頁面上尋找特定詞句。


In [13]:
@ReAct.add_method
def lookup(self, phrase: str, context_length=200):
    """Searches for the `phrase` in the lastest Wikipedia search page
    and returns number of sentences which is controlled by the
    `context_length` parameter.

    Args:
        phrase: Lookup phrase to search for within a page. Generally
        attributes to some specification of any topic.

        context_length: Number of words to consider
        while looking for the answer.

    Returns:
        result: Context related to the `phrase` within the page.
    """
    # get the last searched Wikipedia page and find `phrase` in it.
    page = wikipedia.page(self._search_history[-1], auto_suggest=False)
    page = page.content
    page = self.clean(page)
    start_index = page.find(phrase)

    # extract sentences considering the context length defined
    result = page[max(0, start_index - context_length):start_index+len(phrase)+context_length]
    print(f"Information Source: {self._search_urls[-1]}")
    return result

#### 完畢
指示管道終止執行命令。


In [14]:
@ReAct.add_method
def finish(self, _):
  """Finishes the conversation on encountering <finish> token by
  setting the `self.should_continue_prompting` flag to `False`.
  """
  self.should_continue_prompting = False
  print(f"Information Sources: {self._search_urls}")

### 整合工具


現在你已完成函式定義，下一步是指示模型在發出任何動作 token 時中斷其執行。你將使用 [`genai.GenerativeModel.GenerationConfig`](https://ai.google.dev/api/python/google/generativeai/GenerationConfig) 類別中的 `stop_sequences` 參數來指示模型何時停止。在遇到動作 token 時，你的系統將會擷取最後一個動作和參數，然後呼叫適當的工具。

來自函式的回應將會新增至提示，以繼續此作業流程。


In [15]:
@ReAct.add_method
def __call__(self, user_question, max_calls: int=8, **generation_kwargs):
  """Starts multi-turn conversation with the chat models with function calling

  Args:
      max_calls: max calls made to the model to get the final answer.

      generation_kwargs: Same as genai.GenerativeModel.GenerationConfig
              candidate_count: (int | None) = None,
              stop_sequences: (Iterable[str] | None) = None,
              max_output_tokens: (int | None) = None,
              temperature: (float | None) = None,
              top_p: (float | None) = None,
              top_k: (int | None) = None

  Raises:
      AssertionError: if max_calls is not between 1 and 8
  """

  # hyperparameter fine-tuned according to the paper
  assert 0 < max_calls <= 8, "max_calls must be between 1 and 8"

  if len(self.chat.history) == 0:
    model_prompt = self.prompt.format(question=user_question)
  else:
    model_prompt = user_question

  # stop_sequences for the model to immitate function calling
  callable_entities = ['</search>', '</lookup>', '</finish>']

  generation_kwargs.update({'stop_sequences': callable_entities})

  self.should_continue_prompting = True
  for idx in range(max_calls):

    self.response = self.chat.send_message(content=[model_prompt],
              generation_config=generation_kwargs, stream=True)

    for chunk in self.response:
      print(chunk.text, end=' ')

    response_cmd = self.chat.history[-1].parts[-1].text

    try:
      # regex to extract <function name writen in between angular brackets>
      cmd = re.findall(r'<(.*)>', response_cmd)[-1]
      print(f'</{cmd}>')
      # regex to extract param
      query = response_cmd.split(f'<{cmd}>')[-1].strip()
      # call to appropriate function
      observation = self.__getattribute__(cmd)(query)

      if not self.should_continue_prompting:
        break

      stream_message = f"\nObservation {idx + 1}\n{observation}"
      print(stream_message)
      # send function's output as user's response
      model_prompt = f"<{cmd}>{query}</{cmd}>'s Output: {stream_message}"

    except (IndexError, AttributeError) as e:
      model_prompt = "Please try to generate thought-action-observation traces \
          as instructed by the prompt."

### 測試 ReAct 指令的 Gemini 範例


In [20]:
gemini_ReAct_chat = ReAct(model='gemini-pro', ReAct_prompt='model_instructions.txt')
# Note: try different combinations of generational_config parameters for variational results
gemini_ReAct_chat("What is the total of ages of the main trio from the new Percy Jackson and the Olympians TV series in real life?", temperature=0.2)

Thought 1
I need to search the main trio from the new Percy Jackson  and the Olympians TV series, find their ages in real life, then sum them up.

Action 1
<search>Percy Jackson and the Olymp ians TV series </search>

Observation 1
Could not find ["Percy Jackson and the Olympians TV series"]. Similar: ['Percy Jackson and the Olympians (TV series)', 'Percy Jackson & the Olympians', 'Percy Jackson (film series)', 'Percy Jackson & the Olympians: The Lightning Thief', 'Percy Jackson (disambiguation)', 'Percy Jackson', 'List of characters in mythology novels by Rick Riordan', 'The Lightning Thief', 'The Heroes of Olympus', 'Walker Scobell']. You should search for one of those instead.
Thought 2
I can search Percy Jackson and the Olympians (TV series ) instead.

Action 2
<search>Percy Jackson and the Olympians (TV series) </search>
Information Source: https://en.wikipedia.org/wiki/Percy_Jackson_and_the_Olympians_(TV_series)

Observation 2
Percy Jackson and the Olympians is an American fantas

現在，嘗試在沒有 ReAct 提示的情況下向 `gemini-pro` 模型詢問相同的問題。


In [22]:
gemini_ReAct_chat.model.generate_content("What is the total of ages of the main trio from the new Percy Jackson and the Olympians TV series in real life?").text

'The TV series has not yet been released, so the real-life ages of the main trio are not yet known.'

## 總結

由 ReAct 提示的 Gemini 模型建立於外部資訊來源，因此較不容易出現幻覺。此外，模型產生的 **Thought-Action-Observation** 軌跡透過讓使用者見證模型回答使用者提問的推論過程，進而提升人類的可解釋性和可信度。


## 進一步閱讀


前往 [Streamlit 應用程式](https://mayochat.streamlit.app/) 與 ReAct 提示的 Gemini 機器人互動。
